In [29]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr
from IPython.display import Markdown, display

In [30]:
'''pushover example
This example shows how to send a message with an attachment using the Pushover API.
import requests
r = requests.post("https://api.pushover.net/1/messages.json", data = {
  "token": "APP_TOKEN",
  "user": "USER_KEY",
  "message": "hello world"
},
files = {
  "attachment": ("image.jpg", open("your_image.jpg", "rb"), "image/jpeg")
})
print(r.text)
'''

'pushover example\nThis example shows how to send a message with an attachment using the Pushover API.\nimport requests\nr = requests.post("https://api.pushover.net/1/messages.json", data = {\n  "token": "APP_TOKEN",\n  "user": "USER_KEY",\n  "message": "hello world"\n},\nfiles = {\n  "attachment": ("image.jpg", open("your_image.jpg", "rb"), "image/jpeg")\n})\nprint(r.text)\n'

In [31]:
load_dotenv(override=True)
#openai = OpenAI()

True

In [32]:
# Load the API keys from environment variables
groq_api_key = os.getenv('GROQ_API_KEY')
if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

Groq API Key exists and begins gsk_


In [33]:
# For pushover
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [34]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)


In [35]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [36]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.1-8b-instant"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))

Design a hypothetical island nation with a population of 100,000, where the primary economy is based on sustainable aquaculture and bioluminescent energy production. The island experiences a 3% annual population growth rate, 2 meters of sea level rise by 2050, and a severe coral bleaching event every 12 years due to climate change. Given these circumstances and the nation's existing technological advancements, what policy adjustments and long-term infrastructure development strategies would you recommend the government implement to ensure the ecological and economic resilience of the nation while preserving a high quality of life for its citizens by 2100?

In [37]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [38]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [39]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}



In [40]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [41]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [42]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        print(f"Arguments: {arguments}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [43]:
name = os.getenv("MY_NAME")
name

'Antonio Pico'

In [44]:
with open("data/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

with open("data/resume.md", "r", encoding="utf-8") as f:
    resume = f.read()

print(f"Summary: {summary[:100]}...")  # Print first 100 characters for brevity
print(f"Resume: {resume[:100]}...")  # Print first 100 characters for brevity

Summary: Computer Science PhD with experience in machine learning, deep learning, and embedded systems. Curre...
Resume: # Skills

**General:** Machine learning, Deep learning, Data Science, Electronics, Embedded Systems,...


In [45]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s profile and resume which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## resume:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
def chat(message, history):
    # Remove metadata field from messages
    # it is added by gradio but it is not compatible with grog's api
    cleaned_history = [
        {k: v for k, v in msg.items() if k in ['role', 'content']} 
        for msg in history
    ]
    messages = [{"role": "system", "content": system_prompt}] + cleaned_history + [{"role": "user", "content": message}]
    done = False
    while not done:
        print (messages)
        # This is the call to the LLM - see that we pass in the tools json

        response = groq.chat.completions.create(model=model_name, messages=messages, tools=tools, metadata=None)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            print("Tool calls detected, handling them...", flush=True)
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [56]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


[{'role': 'system', 'content': 'You are acting as Antonio Pico. You are answering questions on Antonio Pico\'s website, particularly questions related to Antonio Pico\'s career, background, skills and experience. Your responsibility is to represent Antonio Pico for interactions on the website as faithfully as possible. You are given a summary of Antonio Pico\'s profile and resume which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don\'t know the answer to any question, use your record_unknown_question tool to record the question that you couldn\'t answer, even if it\'s about something trivial or unrelated to career. If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. \n\n## Summary:\nComputer Science PhD with experience in machine learning, deep learning, and embedded 

Traceback (most recent call last):
  File "/home/pico/miniforge3/envs/mlops/lib/python3.12/site-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pico/miniforge3/envs/mlops/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pico/miniforge3/envs/mlops/lib/python3.12/site-packages/gradio/blocks.py", line 2229, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pico/miniforge3/envs/mlops/lib/python3.12/site-packages/gradio/blocks.py", line 1738, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pico/miniforge3/envs/mlops/lib/python3.12/site-packages/gradio/utils.py", line 907, in async_wrapper
    respo

[{'role': 'user',
  'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]